In [ ]:
# This magic makes plots appear 
%matplotlib inline
import matplotlib.pyplot as plt

import os
os.environ["GADOPT_LOGLEVEL"] = "WARNING"

# Load Firedrake on Colab
try:
    import firedrake
except ImportError:
    !wget "https://github.com/g-adopt/tutorials/releases/latest/download/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

# 2-D Annulus Case
## Weak formulation
As outlined in the previous tutorial, the weak formulation of the equations governing mantle dynamics are:<br>

$$\int_\Omega (\nabla \vec{v})\colon \mu \left[ \nabla \vec{u} + \left( \nabla \vec{u} \right)^T\right] \ dx 
 - \int_{\Omega} \left( \nabla \cdot \vec{v}\right)\ p \ dx
 - \int_{\Omega} Ra_0\ T\ \vec{v}\cdot\hat{k} \ dx = 0 \ \text{ for all } v\in V,$$

$$ \int_\Omega w \nabla \cdot \vec{u} \ dx\ \text{ for all } v\in V,$$

$$  \int_\Omega q\frac{\partial T}{\partial t} \ dx
  + \int_\Omega q \vec{u}\cdot \nabla T \ dx 
  + \int_\Omega \left(\nabla q\right) \cdot \left(\kappa \nabla T\right) \ dx = 0   \text{ for all } q\in Q.$$

This example analyses mantle flow, subject to these same governing equations, in a 2-D annulus domain. We define our domain by the radii of the inner ($r_{\text{min}}$) and outer ($r_{\text{max}}$) boundaries. 
These are chosen such that the non--dimensional depth of the mantle, $z = r_{\text{max}} - r_{\text{min}} = 1$, and the ratio of the inner and outer radii, $f=r_{\text{min}} / r_{\text{max}} = 0.55$, 
thus approximating the ratio between the radii of Earth's surface and core-mantle-boundary (CMB). Specifically, we set $r_{\text{min}} = 1.22$ and $r_{\text{max}} = 2.22$.<br>

This example focusses on differences between running simulations in a 2-D annulus and 2-D Cartesian domain. These can be summarised as follows:
\begin{enumerate}
\item The geometry of the problem - i.e. the computational mesh. 
\item The radial direction of gravity (as opposed to the vertical direction in a Cartesian domain)
\item Initialisation of the temperature field.
\item With free-slip boundary conditions on both boundaries, this case incorporates a velocity nullspace, as well as a pressure nullspace.
\end{enumerate}

In [ ]:
from gadopt import *
import numpy as np
import pyvista as pv # Used for plotting vtk output

We first need a mesh. We generate a circular manifold mesh (with 32 elements in this example) and extrude in the radial direction, using the optional keyword argument `extrusion\_type`, forming 8 layers. To better represent the curvature of the domain and ensure accuracy of our quadratic representation of velocity, we approximate the curved cylindrical shell domain quadratically, using the optional keyword argument `degree`$=2$.

In [ ]:
# Set up geometry:
rmin, rmax, ncells, nlayers = 1.22, 2.22, 32, 8

mesh1d = CircleManifoldMesh(ncells, radius=rmin, degree=2)
mesh = ExtrudedMesh(mesh1d, layers=nlayers, extrusion_type='radial')

bottom_id, top_id = "bottom", "top"
n = FacetNormal(mesh)
domain_volume = assemble(1 * dx(domain=mesh))

### Exercise 7.1:
- Visualise the generated mesh.

- Change the number of radial layers (nlayers) and the lateral resolution (ncells) and visualise the mesh again.

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
File("mesh.pvd").write(Function(V))
mesh_data = pv.read("mesh_0.vtu")
edges = mesh_data.extract_all_edges()

plotter = pv.Plotter(notebook=True)
plotter.add_mesh(edges, color="black")
plotter.camera_position = "xy"

plotter.show(jupyter_backend="static", interactive=False)

We now need the function spaces associated with this mesh, test functions and functions to hold our solutions. We follow the same definitions as our previous tutorial, using the bilinear Q2Q1 element pair for velocity and pressure and Q2 discretisation for temperature:

In [ ]:
V = VectorFunctionSpace(mesh, "CG", degree=2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", degree=1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", degree=2)  # Temperature function space (scalar)
Z = MixedFunctionSpace([V, W])  # Mixed function space
v, w = TestFunctions(Z)
q = TestFunction(Q)
z = Function(Z)  # a field over the mixed function space Z.
u, p = split(z)  # returns symbolic UFL expression for u and p
T = Function(Q, name="Temperature")


We choose the initial temperature distribution to trigger upwelling of 4 equidistant plumes. This initial temperature field is prescribed as:
$$T(x,y) = (r_{\text{max}} - r) + A\cos(4 \; atan2\ (y,x))  \sin(r-r_{\text{min}}) \pi)$$

where $A=0.02$ is the amplitude of the initial perturbation. 

In [ ]:
# Set up temperature field and initialise:
X = SpatialCoordinate(mesh)
r = sqrt(X[0]**2 + X[1]**2)
T.interpolate(rmax - r + 0.02*cos(4*atan_2(X[1], X[0])) * sin((r - rmin) * pi))

We can now visualise this initial temperature field:

In [ ]:
File("temp.pvd").write(T)
temp_data = pv.read("temp_0.vtu")
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(temp_data)
plotter.camera_position = "xy"

plotter.show(jupyter_backend="static", interactive=False)

The Rayleigh number for this problem is defined in addition to the initial timestep $\Delta t$. The viscosity and thermal diffusivity are left at their default values (both = 1). 
These constants are used to create an *Approximation* representing the physical setup of the problem (options include Boussinesq, Extended Boussinesq, Truncated Anelastic Liquid 
and Anelastic Liquid), and a *Timestep Adaptor*, for controlling the time-step length (via a CFL criterion) as the simulation advances in time. 

In [ ]:
Ra, delta_t = Constant(1e5), Constant(1e-6)

approximation = BoussinesqApproximation(Ra)
t_adapt = TimestepAdaptor(delta_t, V, maximum_timestep=0.1, increase_tolerance=1.5)

Boundary conditions for temperature are set to $T = 0$ at the surface ($r_{\text{max}}$) and $T = 1$ at the base ($r_{\text{min}}$).
For velocity, we specify free‐slip conditions on both boundaries. We incorporate these <b>weakly</b> through the <i>Nitsche</i> approximation.
This illustrates a key advantage of the G-ADOPT framework: the user only specifies that the normal component of velocity is zero and all required changes are handled under the hood. 

In [ ]:
temp_bcs = {
    bottom_id: {"T": 1.0},
    top_id: {"T": 0.0},
}

stokes_bcs = {
    bottom_id: {"un": 0},
    top_id: {"un": 0},
}

As noted above, with a free-slip boundary condition on both boundaries, one can add an arbitrary rotation of the form $(-y, x)=r\hat{\mathbf{\theta}}$ to the velocity solution (i.e. this case incorporates a velocity nullspace, as well as a pressure nullspace). These lead to null-modes (eigenvectors) for the linear system, rendering the resulting matrix singular. In preconditioned Krylov methods these null-modes must be subtracted from the approximate solution at every iteration. We do that below, setting up a nullspace object as we did in the previous tutorial, albeit speciying the `rotational` keyword argument to be True. Once again, this removes the requirement for a user to configure these options, further simplifying the task of setting up a (valid) geodynamical simulation. 

Given the increased computational expense (typically requiring more degrees of freedom) in a 2-D annulus domain, the G-ADOPT library defaults to iterative solver parameters (in 2-D Cartesian domains, the framework defaults to direct solver parameters). Our iterative solver setup is configured to use the GAMG preconditioner for the velocity block of the Stokes system, to which we must provide near-nullspace information, which consists of three rotational (`x_rotV`, `y_rotV`, `z_rotV`) and three translational (`nns_x`, `nns_y`, `nns_z`) modes (see Davies et al. (GMD, 2022) for details). 

In [ ]:
Z_nullspace = create_stokes_nullspace(Z, closed=True, rotational=True)
Z_near_nullspace = create_stokes_nullspace(Z, closed=False, rotational=True, translations=[0, 1])

We next come to solving the variational problem, with solver objects for the energy and Stokes systems created. For the energy system we pass in the solution field T, velocity u, the physical approximation, time step, temporal discretisation approach 
(i.e. implicit middle point, being equivalent to a Crank Nicholson scheme) and boundary conditions. For the Stokes system, we pass in the solution fields z, Temperature, the physical approximation, boundary condition, the nullspace and near nullspace objects. We also set the
optional cartesian keyword argument to False, which ensures that the unit vector, $\hat{\vec{k}}$, points radially, in the direction opposite to gravity. Solution of the two variational problems is undertaken by PETSc. 

In [ ]:
energy_solver = EnergySolver(T, u, approximation, delta_t, ImplicitMidpoint, bcs=temp_bcs)
stokes_solver = StokesSolver(z, T, approximation, bcs=stokes_bcs,
                             cartesian=False,
                             nullspace=Z_nullspace, transpose_nullspace=Z_nullspace,
                             near_nullspace=Z_near_nullspace)

We can now initiate the time-loop, with the Stokes and energy systems solved seperately. These `solve` calls once again convert symbolic mathematics into computation. In the time loop, set here to run for 2000 time-steps, we compute the RMS velocity and surface Nusselt number (using definitions from Jarvis, 1993) for diagnostic purposes, and print these results every 50 timesteps.

In [ ]:
no_timesteps = 2000
time = 0

for timestep in range(0, no_timesteps):
    dt = t_adapt.update_timestep(u)
    time += dt

    stokes_solver.solve()
    energy_solver.solve()

    # Compute diagnostics:
    u_rms = sqrt(assemble(dot(u, u) * dx)) * sqrt(1./domain_volume)
    f_ratio = rmin/rmax
    top_scaling = -1.3290170684486309  # log(f_ratio) / (1.- f_ratio)
    bot_scaling = -0.7303607313096079  # (f_ratio * log(f_ratio)) / (1.- f_ratio)
    nusselt_number_top = (assemble(dot(grad(T), n) * ds_t) / assemble(1 * ds_t(domain=mesh))) * top_scaling
    nusselt_number_base = (assemble(dot(grad(T), n) * ds_b) / assemble(1 * ds_b(domain=mesh))) * bot_scaling

    if timestep % 50 == 0:
        print(f"{u_rms=}, Nu_t={nusselt_number_top}, Nu_b={nusselt_number_base}, {maxchange=}")

We can now visualise the final temperature field:

In [ ]:
File("temp.pvd").write(T)
temp_data = pv.read("temp_0.vtu")
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(temp_data)
plotter.camera_position = "xy"

plotter.show(jupyter_backend="static", interactive=False)

### Exercise 7.2
- Change the top boundary conditions from free-slip to no-slip (zero velocity). Note that with zero-slip boundary conditions on each boundary, there is no velocity `nullspace` for this problem. 